In [3]:
import sys
from env_key_manager import APIKeyManager

# 创建实例
key_manager = APIKeyManager()

# 设置环境变量
key_manager.setup_api_key(["DEEPSEEK_API_KEY"])

# 查看Python版本
!python -V
# 查看安装的库
if 'win' in sys.platform.lower():
    !pip list | findstr "lang openai llm tiktoken chromadb cryptography duck unstructured numpy scipy"
else:
    !pip list | grep -E 'lang|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.10.16
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        6.3.7
langchain                                0.3.19
langchain-chroma                         0.2.2
langchain-community                      0.3.18
langchain-core                           0.3.49
langchain-deepseek                       0.1.3
langchain-openai                         0.3.11
langchain-text-splitters                 0.3.6
langgraph                                0.3.21
langgraph-checkpoint                     2.0.23
langgraph-prebuilt                       0.1.7
langgraph-sdk                            0.1.60
langserve                                0.3.1
langsmith                                0.3.8
numpy                                    1.26.4
openai                                   1.69.0
scipy                                    1.15.2
tiktoken                                 0.9.0


[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/tagging.ipynb)

# 将文本分类为标签

标记是指为文档添加类别标签，例如：

- 情绪
- 语言
- 风格（正式、非正式等）
- 涵盖主题
- 政治倾向

![图片描述](../../static/img/tagging.png)

## 概述

标签功能包含以下几个组成部分：

* `function`（函数）：与[提取](/docs/tutorials/extraction)类似，标注功能通过[函数](https://openai.com/blog/function-calling-and-other-api-updates)来指定模型应如何对文档进行标注
* `schema`: 定义我们想要如何标记文档

## 快速入门

让我们来看一个非常直接的示例，展示如何在LangChain中使用OpenAI的工具调用来进行标记。我们将使用OpenAI模型支持的[`with_structured_output`](/docs/how_to/structured_output)方法。

In [ ]:
# pip install --upgrade --quiet langchain-core

我们需要加载一个[聊天模型](/docs/integrations/chat/)：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [4]:
# | output: false
# | echo: false

from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(model="deepseek-chat")

让我们在模式中指定一个带有若干属性及其预期类型的Pydantic模型。

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
从以下的文章中提取所需的信息。

只提取'Classification'函数中提到的属性。

文章：
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="文本的情感")
    aggressiveness: int = Field(
        description="文本的侵略性，按照1到10的比例"
    )
    language: str = Field(description="文本所用的语言")


# LLM
llm = ChatDeepSeek(temperature=0, model="deepseek-chat").with_structured_output(
    Classification
)

In [6]:
inp = "我非常高兴认识你！我认为我们将成为非常好的朋友！"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='非常高兴', aggressiveness=1, language='中文')

如果我们想要字典形式的输出，只需调用 `.model_dump()` 即可

In [7]:
inp = "我非常生气你！我会给你应得的惩罚！"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

{'sentiment': '生气', 'aggressiveness': 8, 'language': 'zh'}

从这些例子中可以看出，它准确理解了我们想要表达的内容。

结果各不相同，因此我们可能会得到不同语言的情感表达（例如“positive”、“enojado”等）。

我们将在下一节中了解如何控制这些结果。

## 更精细的控制

谨慎的模式定义让我们能更好地控制模型的输出。

具体而言，我们可以定义：

- 每个属性的可能取值
- 描述以确保模型理解该属性
- 需要返回的必要属性

让我们重新声明我们的Pydantic模型，通过枚举类型来控制前面提到的每个方面：

In [8]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["高兴", "中立", "悲伤"])
    aggressiveness: int = Field(
        ...,
        description="描述语句的攻击性，数字越高攻击性越强",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["西班牙语", "英语", "法语", "德语", "意大利语"]
    )

In [10]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
从以下的文章中提取所需的信息。

只提取'Classification'函数中提到的属性。

文章：
{input}
"""
)

llm = ChatDeepSeek(temperature=0, model="deepseek-chat").with_structured_output(
    Classification
)

现在答案将按照我们预期的方式受到限制！

In [11]:
inp = "我非常高兴认识你！我相信我们会成为很好的朋友！"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='高兴', aggressiveness=1, language='西班牙语')

In [12]:
inp = "我非常生气！我会让你付出代价的！"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='悲伤', aggressiveness=5, language='西班牙语')

In [13]:
inp = "这里的天气还不错，我可以只穿一件外套就出门了"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='高兴', aggressiveness=1, language='西班牙语')

[LangSmith 追踪记录](https://smith.langchain.com/public/38294e04-33d8-4c5a-ae92-c2fe68be8332/r) 让我们得以一窥究竟：

![图片描述](../../static/img/tagging_trace.png)

### 深入探索

* 您可以使用[元数据标记器](/docs/integrations/document_transformers/openai_metadata_tagger)文档转换器从LangChain `Document`中提取元数据。
* 这与标记链（tagging chain）的基本功能相同，只是应用于 LangChain 的 `Document`。